In [11]:
import numpy as np
import pandas as pd
import math
import astropy.constants as const
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Parameters that you can change for new model
c = const.c.cgs.value #speed of light in cm/s
c_factor = 0.5 #factor to change the speed of light for the velocity of forward shock (needs to be above 0.7 for a_gamma_sf to be positive)
v_fs = c_factor * c #velocity of forward shock in cm/s
dens = 1e-16 #density of agn disk
#do 1e-15, 1e-9, 1e-12

b_fs = v_fs / c #page 2 bottom right
b_sf = 0.5 #seen on page 5
e_e = 0.3 #page 8 top right
e_b = 0.1 #page 8 top right 
xi = 1 # given on page 7 top left
p = 2.5 #foud on page 8 top right
gamma_fs = 1 / np.sqrt(1 - b_fs**2)
gamma_fs_f = gamma_fs**(1 + np.sqrt(3)) #page 5 top left
gamma_sf = gamma_fs / np.sqrt(2) #page 5 top left
gamma_sf_f = gamma_sf**(1 + np.sqrt(3))
a_gamma_sf = (gamma_sf-1)*(4*gamma_sf+3) #left of eq 24
gamma_m = 40 * (e_e/0.3)*((gamma_sf-1)/0.2) #used in eq 25 from eq 16
B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((a_gamma_sf/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17
gamma_a = 60 #used in eq 27 has a messy equation at (21) pls say we dont need it
n = dens / const.m_p

def lum_sh(): #will get from emily
    if b_fs * gamma_fs <=1:
        lum_sh = 42
    else: #b_fs * gamma_fs > 1
        lum_sh = 44
    return lum_sh

def lum_syn(): #eq 15
    Yssc = (e_e / e_b)**(1/3) #page 8 top left
    Y2ndIC = (e_e / e_b)**(2/3) #page 8 top left
    f_dilution = 1 #page 6 bottom left
    if b_fs * gamma_fs <= 1:
        f_beaming = 1
    else: #b_fs * gamma_fs > 1
        f_beaming = 2 * (gamma_sf_f**2)
    lum_syn = e_e * f_dilution * f_beaming * (lum_sh() / (1 + Yssc + Y2ndIC))
    return lum_syn

freq = (1e17) #frequency range to test in Hz

def gamma_a_big(): #when gamma_a > gamma_m
    q = 2.5 #need to set it to 2 when gamma_a > gamma_m
    gamma_2 = gamma_max #set to gamma_m when gamma_a > gamma_m
    gamma_1 = gamma_m #set to gamma_max when gamma_a > gamma_m
    pa = np.sin(np.pi/2) #set to 90 degrees for the pitch angle
    rho_sf = 1 #placeholder for now need to hear from Hiromichi
    
    tc_gm = 3*10 * ((e_b/0.1)**-1)* ((((gamma_sf-1)*(4*gamma_sf+3))/1.5)**-1) * ((dens/1e-16)**-1) * ((gamma_sf/1.2)**-1) * ((gamma_m/40)**-1) #eq 20, the last term needs to be changed to gamma_a for big one
    delta_shell_m = tc_gm * v_fs #need to make this to delta shell gamma_a for the next calculation
    tq_gm = (np.pi/3*np.sqrt(3)) * ((q**2 + q -2)*(gamma_1**-5)/1-((gamma_2/gamma_1)**(-q+1))) * ((np.exp(1)*rho_sf*delta_shell_m)/(B_sf*pa*const.m_p.cgs.value)) #eq 22 need to change delta shell to delta shell gamma a
    cq_1 = ((2**((q+1)/2))/((q+1)+1)) * math.gamma((q+1/4)-(1/12)) * math.gamma((q+1/4)+(19/12)) #eq 23 for Cq+1
    gamma_a0 = gamma_m * ((tq_gm * cq_1)**(1/(q+4))) #eq 21 to start not sure how to make it

    tc_ga = 3*10 * ((e_b/0.1)**-1)* ((((gamma_sf-1)*(4*gamma_sf+3))/1.5)**-1) * ((dens/1e-16)**-1) * ((gamma_sf/1.2)**-1) * ((gamma_a/60)**-1) #eq 20 edited to use gamma_a instead of gamma_m
    delta_shell_a = tc_ga * v_fs #reuse this for gamma_a need to make it into a loop
    tq_ga = (np.pi/3*np.sqrt(3)) * ((q**2 + q -2)*(gamma_1**-5)/1-((gamma_2/gamma_1)**(-q+1))) * ((np.exp(1)*rho_sf*delta_shell_a)/(B_sf*pa*const.m_p.cgs.value)) #eq 22 need to make into loop
    gamma_a1 = gamma_m * ((tq_ga * cq_1)**(1/(q+4)))
    return gamma_a1

def gamma_a_small(): #when gamma_m > gamma_a
    q = 2
    gamma_2 = gamma_m
    gamma_1 = gamma_max
    pa = np.sin(np.pi/2) #set to 90 degrees for the pitch angle
    rho_sf = 1 #placeholder for now need to hear from Hiromichi
    
    tc_gm = 3*10 * ((e_b/0.1)**-1)* ((((gamma_sf-1)*(4*gamma_sf+3))/1.5)**-1) * ((dens/1e-16)**-1) * ((gamma_sf/1.2)**-1) * ((gamma_m/40)**-1) #eq 20, the last term needs to be changed to gamma_a for big one
    delta_shell_m = tc_gm * v_fs #need to make this to delta shell gamma_a for the next calculation
    tq_gm = (np.pi/3*np.sqrt(3)) * ((q**2 + q -2)*(gamma_1**-5)/1-((gamma_2/gamma_1)**(-q+1))) * ((np.exp(1)*rho_sf*delta_shell_m)/(B_sf*pa*const.m_p.cgs.value)) #eq 22 need to change delta shell to delta shell gamma a
    cq_1 = ((2**((q+1)/2))/((q+1)+1)) * math.gamma((q+1/4)-(1/12)) * math.gamma((q+1/4)+(19/12)) #eq 23 for Cq+1
    gamma_a0 = gamma_m * ((tq_gm * cq_1)**(1/(q+4))) #eq 21 to start not sure how to make it

    tc_ga = 3*10 * ((e_b/0.1)**-1)* ((((gamma_sf-1)*(4*gamma_sf+3))/1.5)**-1) * ((dens/1e-16)**-1) * ((gamma_sf/1.2)**-1) * ((gamma_a/60)**-1) #eq 20 edited to use gamma_a instead of gamma_m
    delta_shell_a = tc_ga * v_fs #reuse this for gamma_a need to make it into a loop
    tq_ga = (np.pi/3*np.sqrt(3)) * ((q**2 + q -2)*(gamma_1**-5)/1-((gamma_2/gamma_1)**(-q+1))) * ((np.exp(1)*rho_sf*delta_shell_a)/(B_sf*pa*const.m_p.cgs.value)) #eq 22 need to make into loop
    gamma_a2 = gamma_m * ((tq_ga * cq_1)**(1/(q+4)))

    return gamma_a2


def lum_tot(freq):
    freq_sync = 2e9 * (gamma_sf/1.2) * (B_sf/600) #eq 24
    freq_m = 3e12*((gamma_m/40)**2)*(freq_sync/2e9) #eq 25
    freq_max = 2e21*((gamma_max/1e6)**2)*(freq_sync/2e9) #eq 26 (need to ask if i use the first or second equation)
    freq_a = 7e12 * ((gamma_a/60)**2) * (freq_sync/2e9) #eq 27
    freq_combo = 0
    
    if freq_a < freq_m: #eq 28
        if freq_m < freq < freq_max:
            freq_combo = (freq/freq_m)**(-(p-1)/2)
        elif freq_a < freq < freq_m:
            freq_combo = (freq/freq_m)**(0.5)
        elif freq_sync < freq < freq_a:
            freq_combo = ((freq/freq_a)**(3))*((freq_a/freq_m)**(0.5))
        else:
            freq_combo = 0.1
    if freq_m < freq_a: #eq 29
        if freq_a < freq < freq_max:
            freq_combo = (freq/freq_a)**(-(p-1)/2) + np.exp(1-(freq/freq_a)**(0.5))
        elif freq < freq_a:
            freq_combo = (freq/freq_a)**3
        else:
            freq_combo = 0

    lum_tot = (lum_syn() * freq_combo)/freq
    return lum_tot

test = lum_tot(freq)

print( gamma_m, gamma_sf, gamma_a_big(), gamma_a_small())
print("xi =", xi)
print("a_gamma_sf =", a_gamma_sf)
print("e_b =", e_b)
print("dens =", dens)
print("b_sf =", b_sf)

-36.70068381445479 0.816496580927726 nan nan
xi = 1
a_gamma_sf = -1.1498299142610593
e_b = 0.1
dens = 1e-16
b_sf = 0.5


/var/folders/pw/nc5rj5md667b29kntncq7jbw0000gn/T/ipykernel_16330/2743637605.py:27: RuntimeWarning: invalid value encountered in scalar power
  B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
/var/folders/pw/nc5rj5md667b29kntncq7jbw0000gn/T/ipykernel_16330/2743637605.py:28: RuntimeWarning: invalid value encountered in scalar power
  gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((a_gamma_sf/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17


In [15]:

gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((1.5/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17

print(a_gamma_sf, gamma_sf)

-1.1498299142610593 0.816496580927726
